# SentencePiece and BPE 

## Introduction to Tokenization

In order to process text in neural network models it is first required to **encode** text as numbers with ids, since the tensor operations act on numbers. Finally, if the output of the network is to be words, it is required to **decode** the predicted tokens ids back to text.

To encode text, the first decision that has to be made is to what level of granularity are we going to consider the text? Because ultimately, from these **tokens**, features are going to be created about them. Many different experiments have been carried out using *words*, *morphological units*, *phonemic units* or *characters* as tokens. For example, 

- Tokens are tricky. (raw text)
- Tokens are tricky . ([words](https://arxiv.org/pdf/1301.3781))
- Token s _ are _ trick _ y . ([morphemes](https://arxiv.org/pdf/1907.02423.pdf))
- t oʊ k ə n z _ ɑː _ ˈt r ɪ k i. ([phonemes](https://www.aclweb.org/anthology/W18-5812.pdf), for STT)
- T o k e n s _ a r e _ t r i c k y . ([character](https://www.aclweb.org/anthology/C18-1139/))

But how to identify these units, such as words, is largely determined by the language they come from. For example, in many European languages a space is used to separate words, while in some Asian languages there are no spaces between words. Compare English and Mandarin.

- Tokens are tricky. (original sentence)
- 标记很棘手 (Mandarin)
- Biāojì hěn jíshǒu (pinyin)
- 标记 很 棘手 (Mandarin with spaces)


So, the ability to **tokenize**, i.e. split text into meaningful fundamental units, is not always straight-forward.

Also, there are practical issues of how large our *vocabulary* of words, `vocab_size`, should be, considering memory limitations vs. coverage. A compromise may be need to be made between: 
* the finest-grained models employing characters which can be memory intensive and 
* more computationally efficient *subword* units such as [n-grams](https://arxiv.org/pdf/1712.09405) or larger units.

In [SentencePiece](https://www.aclweb.org/anthology/D18-2012.pdf) unicode characters are grouped together using either a [unigram language model](https://www.aclweb.org/anthology/P18-1007.pdf) (used in this week's assignment) or [BPE](https://arxiv.org/pdf/1508.07909.pdf), **byte-pair encoding**. We will discuss BPE, since BERT and many of its variants use a modified version of BPE and its pseudocode is easy to implement and understand... hopefully!

## SentencePiece Preprocessing
### NFKC Normalization

Unsurprisingly, even using unicode to initially tokenize text can be ambiguous, e.g., 

In [1]:
eaccent = '\u00E9'
e_accent = '\u0065\u0301'

print(f'{eaccent} = {e_accent} : {eaccent == e_accent}')

é = é : False


SentencePiece uses the Unicode standard normalization form, [NFKC](https://en.wikipedia.org/wiki/Unicode_equivalence), so this isn't an issue. Looking at the example from above but with normalization:

In [2]:
from unicodedata import normalize

norm_eaccent = normalize('NFKC', '\u00E9')
norm_e_accent = normalize('NFKC', '\u0065\u0301')

print(f'{norm_eaccent} = {norm_e_accent} : {norm_eaccent == norm_e_accent}')

é = é : True


Normalization has actually changed the unicode code point (unicode unique id) for one of these two characters.

In [3]:
def get_hex_encoding(s):
    """Convert each character in the string `s` to its hexadecimal representation."""
    # For each character `c` in the string `s`, get its Unicode code point using `ord(c)` and 
    # convert it to a hexadecimal string using `hex()`
    # Join these hexadecimal representations with a space separator
    return ' '.join(hex(ord(c)) for c in s)

def print_string_and_encoding(s):
    """Print the string `s` along with its hexadecimal encoding."""
    # Call `get_hex_encoding(s)` to get the hexadecimal representation of each character in the string `s`
    # Print the original string `s` and its hexadecimal encoding
    print(f'{s} : {get_hex_encoding(s)}')

In [4]:
for s in [eaccent, e_accent, norm_eaccent, norm_e_accent]:
    print_string_and_encoding(s)

é : 0xe9
é : 0x65 0x301
é : 0xe9
é : 0xe9


This normalization has other side effects which may be considered useful such as converting curly quotes &ldquo; to " their ASCII equivalent. (<sup>*</sup>Although we *now* lose directionality of the quote...)

### Lossless Tokenization

SentencePiece also ensures that when you tokenize your data and detokenize your data the original position of white space is preserved. However, tabs and newlines are converted to spaces.

To ensure this **lossless tokenization**, SentencePiece replaces white space with _ (U+2581). So that a simple join of the tokens by replacing underscores with spaces can restore the white space, even if there are consecutive symbols. But remember first to normalize and then replace spaces with _ (U+2581).

In [5]:
s = 'Tokenization is hard.'
sn = normalize('NFKC', s)
sn_ = sn.replace(' ', '\u2581')

In [6]:
print(get_hex_encoding(s))
print(get_hex_encoding(sn))
print(get_hex_encoding(sn_))

0x54 0x6f 0x6b 0x65 0x6e 0x69 0x7a 0x61 0x74 0x69 0x6f 0x6e 0x20 0x69 0x73 0x20 0x68 0x61 0x72 0x64 0x2e
0x54 0x6f 0x6b 0x65 0x6e 0x69 0x7a 0x61 0x74 0x69 0x6f 0x6e 0x20 0x69 0x73 0x20 0x68 0x61 0x72 0x64 0x2e
0x54 0x6f 0x6b 0x65 0x6e 0x69 0x7a 0x61 0x74 0x69 0x6f 0x6e 0x2581 0x69 0x73 0x2581 0x68 0x61 0x72 0x64 0x2e


## BPE Algorithm

After discussing the preprocessing that SentencePiece performs, you will get the data, preprocess it, and apply the BPE algorithm. You will see how this reproduces the tokenization produced by training SentencePiece on the example dataset (from this week's assignment).

### Preparing our Data
First, you get the Squad data and process it as above.

In [7]:
import ast

def convert_json_examples_to_text(filepath):
    example_jsons = list(map(ast.literal_eval, open(filepath))) # Read in the json from the example file
    texts = [example_json['text'].decode('utf-8') for example_json in example_jsons] # Decode the byte sequences
    text = '\n\n'.join(texts)       # Separate different articles by two newlines
    text = normalize('NFKC', text)  # Normalize the text

    with open('example.txt', 'w') as fw:
        fw.write(text)
    
    return text

In [8]:
text = convert_json_examples_to_text('./data/data.txt')
print(text[:900])

Beginners BBQ Class Taking Place in Missoula!
Do you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.
He will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.
The cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.

Discussion in 'Mac OS X Lion (10.7)' started by axboi87, Jan 20, 2012.
I've got a 500gb internal drive and a 240gb SSD.
When trying to restore using di


In the algorithm the `vocab` variable is actually a frequency dictionary of the words. Those words have been prepended with an *underscore* to indicate that they are the beginning of a word. Finally, the characters have been delimited by spaces so that the BPE algorithm can group the most common characters together in the dictionary in a greedy fashion. You will see how that is done shortly.

In [9]:
from collections import Counter

# Create a Counter object for words in `text`, prefixing each word with the Unicode character '\u2581'
vocab = Counter(['\u2581' + word for word in text.split()])

# Convert each word in `vocab` to a space-separated format and rebuild the dictionary
vocab = {' '.join([l for l in word]): freq for word, freq in vocab.items()}

In [10]:
def show_vocab(vocab, end='\n', limit=20):
    
    """Show word frequencies in vocab up to the limit number of words"""  # Function docstring explaining its purpose

    shown = 0                              # Initialize a counter for the number of words shown
    
    for word, freq in vocab.items():       # Iterate over each word and its frequency in the vocab dictionary
        print(f'{word}: {freq}', end=end)  # Print the word and its frequency, ending with the specified end character
        shown += 1                         # Increment the counter
        if shown > limit:                  # If the number of words shown exceeds the limit, break the loop
            break

In [11]:
show_vocab(vocab)

▁ B e g i n n e r s: 1
▁ B B Q: 3
▁ C l a s s: 2
▁ T a k i n g: 1
▁ P l a c e: 1
▁ i n: 15
▁ M i s s o u l a !: 1
▁ D o: 1
▁ y o u: 13
▁ w a n t: 1
▁ t o: 33
▁ g e t: 2
▁ b e t t e r: 2
▁ a t: 1
▁ m a k i n g: 2
▁ d e l i c i o u s: 1
▁ B B Q ?: 1
▁ Y o u: 1
▁ w i l l: 6
▁ h a v e: 4
▁ t h e: 31


You check the size of the vocabulary (frequency dictionary) because this is the one hyperparameter that BPE depends on crucially on how far it breaks up a word into SentencePieces. It turns out that for your trained model on the small dataset that 60% of 455 merges of the most frequent characters need to be done to reproduce the upperlimit of a 32K `vocab_size` over the entire corpus of examples.

In [12]:
print(f'Total number of unique words: {len(vocab)}')

print(f'Number of merges required to reproduce SentencePiece training on the whole corpus: {int(0.60*len(vocab))}')

Total number of unique words: 455
Number of merges required to reproduce SentencePiece training on the whole corpus: 273


### BPE Algorithm
Directly from the BPE paper you have the following algorithm. 

In [27]:
import re, collections


def get_stats(vocab):
    """
    Calculate frequency of adjacent pairs of symbols in the vocabulary.
    
    Args:
        vocab (dict): Dictionary of word frequencies.
        
    Returns:
        pairs (defaultdict): Frequency of each symbol pair.
    """
    pairs = collections.defaultdict(int)   # Create a default dictionary to store the frequency of symbol pairs
    for word, freq in vocab.items():       # Iterate over each word and its frequency in the vocabulary
        symbols = word.split()             # Split the word into a list of symbols (usually characters or subwords)
        for i in range(len(symbols) - 1):  # Loop through symbols to find adjacent pairs
            pairs[symbols[i], symbols[i+1]] += freq  # Increment the count of the symbol pair by the word frequency
    return pairs                                     # Return the dictionary of symbol pairs and their frequencies


def merge_vocab(pair, v_in):
    """
    Merge the most frequent pair of symbols in the vocabulary.
    
    Args:
        pair (tuple): The most frequent symbol pair to be merged.
        v_in (dict): The current vocabulary dictionary.
    
    Returns:
        v_out (dict): Updated vocabulary after merging the symbol pair.
    """
    v_out = {}                                       # Create an empty dictionary for the updated vocabulary
    bigram = re.escape(' '.join(pair))               # Join the pair into a bigram and escape special characters for regex
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')  # Compile a regex pattern to match the bigram as a whole word
    for word in v_in:                                # Iterate over each word in the input vocabulary
        w_out = p.sub(''.join(pair), word)           # Replace the bigram with the merged symbol in the word
        v_out[w_out] = v_in[word]                    # Update the output vocabulary with the merged word
    return v_out                                     # Return the updated vocabulary


def get_sentence_piece_vocab(vocab, frac_merges=0.60):
    """
    Generate a SentencePiece-like vocabulary by performing a specified fraction of merges.
    
    Args:
        vocab (dict): Original vocabulary of word frequencies.
        frac_merges (float): Fraction of merges to perform (default is 0.60).
    
    Returns:
        sp_vocab (dict): Updated vocabulary after merges.
    """
    sp_vocab = vocab.copy()                        # Make a copy of the input vocabulary to avoid modifying the original
    num_merges = int(len(sp_vocab) * frac_merges)  # Calculate the number of merges to perform based on the fraction
    
    for i in range(num_merges):                    # Loop to perform the specified number of merges
        pairs = get_stats(sp_vocab)                # Get the frequency of all symbol pairs in the current vocabulary
        best = max(pairs, key=pairs.get)           # Identify the most frequent symbol pair
        sp_vocab = merge_vocab(best, sp_vocab)     # Merge the most frequent pair in the vocabulary

    return sp_vocab                                # Return the updated vocabulary after the merges

To understand what's going on first take a look at the third function `get_sentence_piece_vocab`. It takes in the current `vocab` word-frequency dictionary and the fraction, `frac_merges`, of the total `vocab_size` to merge characters in the words of the dictionary, `num_merges` times. Then for each *merge* operation it `get_stats` on how many of each pair of character sequences there are. It gets the most frequent *pair* of symbols as the `best` pair. Then it merges that pair of symbols (removes the space between them) in each word in the `vocab` that contains this `best` (= `pair`). Consequently, `merge_vocab` creates a new `vocab`, `v_out`. This process is repeated `num_merges` times and the result is the set of SentencePieces (keys of the final `sp_vocab`).

### Additional Discussion of BPE Algorithm

Please feel free to skip the below if the above description was enough.

In a little more detail you can see in `get_stats` you initially create a list of bigram (two character sequence) frequencies from the vocabulary. Later, this may include trigrams, quadgrams, etc. Note that the key of the `pairs` frequency dictionary is actually a 2-tuple, which is just shorthand notation for a pair.

In `merge_vocab` you take in an individual `pair` (of character sequences, note this is the most frequency `best` pair) and the current `vocab` as `v_in`. You create a new `vocab`, `v_out`, from the old by joining together the characters in the pair (removing the space), if they are present in a word of the dictionary.

[Warning](https://regex101.com/): the expression `(?<!\S)` means that either a whitespace character follows before the `bigram` or there is nothing before the bigram (it is the beginning of the word), similarly for `(?!\S)` for preceding whitespace or the end of the word. 

In [14]:
sp_vocab = get_sentence_piece_vocab(vocab)
show_vocab(sp_vocab) 

▁B e g in n ers: 1
▁BBQ: 3
▁Cl ass: 2
▁T ak ing: 1
▁P la ce: 1
▁in: 15
▁M is s ou la !: 1
▁D o: 1
▁you: 13
▁w an t: 1
▁to: 33
▁g et: 2
▁be t ter: 2
▁a t: 1
▁mak ing: 2
▁d e l ic i ou s: 1
▁BBQ ?: 1
▁ Y ou: 1
▁will: 6
▁have: 4
▁the: 31


## Train SentencePiece BPE Tokenizer on Example Data
### Explore SentencePiece Model
First, explore the SentencePiece model provided with this week's assignment. Remember you can always use Python's built in `help` command to see the documentation for any object or method.

In [15]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor(model_file='./data/sentencepiece.model')

In [16]:
# help(sp)

Try it out on the first sentence of the example text.

In [17]:
s0 = 'Beginners BBQ Class Taking Place in Missoula!'

In [18]:
# encode: text => id
# Convert the input text `s0` into a list of subword tokens (pieces)
print(sp.encode_as_pieces(s0))                    # Output: List of subword pieces (tokens)

# Convert the input text `s0` into a list of corresponding token IDs
print(sp.encode_as_ids(s0))                       # Output: List of token IDs corresponding to the subword pieces

# decode: id => text
# Decode a list of subword pieces back into the original text
print(sp.decode_pieces(sp.encode_as_pieces(s0)))  # Output: Reconstructed text from subword pieces

# Decode a list of token IDs back into the original text
print(sp.decode_ids([12847, 277]))                # Output: Reconstructed text from the given token IDs

['▁Beginn', 'ers', '▁BBQ', '▁Class', '▁', 'Taking', '▁Place', '▁in', '▁Miss', 'oul', 'a', '!']
[12847, 277, 15068, 4501, 3, 12297, 3399, 16, 5964, 7115, 9, 55]
Beginners BBQ Class Taking Place in Missoula!
Beginners


Notice how SentencePiece breaks the words into seemingly odd parts, but you have seen something similar with BPE. But how close was the model trained on the whole corpus of examples with a `vocab_size` of 32,000 instead of 455? Here you can also test what happens to white space, like '\n'. 

But first note that SentencePiece encodes the SentencePieces, the tokens, and has reserved some of the ids as can be seen in this week's assignment.

In [19]:
uid = 15068  # Example token ID
spiece = "\u2581BBQ"  # Example subword piece (prefixed with '\u2581', a special symbol used in SentencePiece)
unknown = "__MUST_BE_UNKNOWN__"  # Example of a text that is not in the vocabulary (unknown token)

# id <=> piece conversion
# Convert a token ID to its corresponding subword piece
print(f'SentencePiece for ID {uid}: {sp.id_to_piece(uid)}')         # Output: Subword piece corresponding to the given ID

# Convert a subword piece to its corresponding token ID
print(f'ID for Sentence Piece {spiece}: {sp.piece_to_id(spiece)}')  # Output: ID corresponding to the given subword piece

# returns 0 for unknown tokens (we can change the ID for UNK)
# Convert an unknown text (not in the vocabulary) to its corresponding token ID, which is typically 0 by default
print(f'ID for unknown text {unknown}: {sp.piece_to_id(unknown)}')  # Output: ID for unknown token (usually 0)

SentencePiece for ID 15068: ▁BBQ
ID for Sentence Piece ▁BBQ: 15068
ID for unknown text __MUST_BE_UNKNOWN__: 2


In [20]:
# Print the ID for the beginning of the sentence token (BOS)
print(f'Beginning of sentence id: {sp.bos_id()}')  # Output: ID used to represent the beginning of a sentence

# Print the ID for the padding token (PAD)
print(f'Pad id: {sp.pad_id()}')      # Output: ID used for padding, typically used to align sequences to the same length

# Print the ID for the end of the sentence token (EOS)
print(f'End of sentence id: {sp.eos_id()}')        # Output: ID used to represent the end of a sentence

# Print the ID for the unknown token (UNK)
print(f'Unknown id: {sp.unk_id()}')  # Output: ID used to represent tokens that are not in the vocabulary

# Print the size of the vocabulary
print(f'Vocab size: {sp.vocab_size()}')            # Output: Total number of tokens (pieces) in the vocabulary

Beginning of sentence id: -1
Pad id: 0
End of sentence id: 1
Unknown id: 2
Vocab size: 32000


You can also check what are the ids for the first part and last part of the vocabulary.

In [21]:
# Print the header for the table displaying ID, SentencePiece, and whether it is a control symbol
print('\nId\tSentP\tControl?')  # Print column headers for ID, SentencePiece, and Control status
print('------------------------')  # Print a separator line for readability

# <unk>, <s>, </s> are defined by default in SentencePiece. Their ids are (0, 1, 2).
# <s> (beginning of sentence) and </s> (end of sentence) are defined as 'control' symbols.

# Iterate through the first 10 IDs to display their corresponding SentencePiece and control status
for uid in range(10):  
    print(uid, sp.id_to_piece(uid), sp.is_control(uid), sep='\t')  # Output: ID, corresponding piece, 
                                                                   # and control status (True/False)

# Uncomment the following code block to iterate over the last 10 IDs in the vocabulary and display their corresponding 
# SentencePiece and control status

# for uid in range(sp.vocab_size()-10, sp.vocab_size()):  
#     print(uid, sp.id_to_piece(uid), sp.is_control(uid), sep='\t')  # Output: ID, corresponding piece, and control status (True/False)


Id	SentP	Control?
------------------------
0	<pad>	True
1	</s>	True
2	<unk>	False
3	▁	False
4	X	False
5	.	False
6	,	False
7	s	False
8	▁the	False
9	a	False


### Train SentencePiece BPE model with our example.txt

Finally, train your own BPE model directly from the SentencePiece library and compare it to the results of the implemention of the algorithm from the BPE paper itself.

In [22]:
# Train a SentencePiece model using BPE (Byte-Pair Encoding) on the input file 'example.txt'
# '--input=example.txt': Specifies the input file containing text data for training
# '--model_prefix=example_bpe': Sets the prefix for the output model files (example_bpe.model and example_bpe.vocab)
# '--vocab_size=450': Defines the size of the vocabulary to be created
# '--model_type=bpe': Specifies the model type to use, which is BPE (Byte-Pair Encoding) in this case
spm.SentencePieceTrainer.train('--input=example.txt --model_prefix=example_bpe --vocab_size=450 --model_type=bpe')

# Initialize a SentencePieceProcessor object to use the trained BPE model
sp_bpe = spm.SentencePieceProcessor()

# Load the trained BPE model from the file 'example_bpe.model'
sp_bpe.load('example_bpe.model')

# Print a header indicating that the following output is for the BPE encoding
print('*** BPE ***')

# Encode the input string `s0` into a list of subword tokens (pieces) using the BPE model
print(sp_bpe.encode_as_pieces(s0))  # Output: List of subword pieces (tokens) obtained using the BPE model

*** BPE ***
['▁B', 'e', 'ginn', 'ers', '▁BBQ', '▁Cl', 'ass', '▁T', 'ak', 'ing', '▁P', 'la', 'ce', '▁in', '▁M', 'is', 's', 'ou', 'la', '!']


sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=example.txt --model_prefix=example_bpe --vocab_size=450 --model_type=bpe
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: example.txt
  input_format: 
  model_prefix: example_bpe
  model_type: BPE
  vocab_size: 450
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_p

In [23]:
# Display the vocabulary stored in `sp_vocab` with word frequencies, using a comma as the separator between entries.
# The `show_vocab` function takes a dictionary (`sp_vocab`) and prints up to a specified limit of word-frequency pairs.
# `end=', '` specifies that the output should use a comma followed by a space to separate each entry.

show_vocab(sp_vocab, end=', ')  # Output: Vocabulary displayed with a comma and space separating each word-frequency pair

▁B e g in n ers: 1, ▁BBQ: 3, ▁Cl ass: 2, ▁T ak ing: 1, ▁P la ce: 1, ▁in: 15, ▁M is s ou la !: 1, ▁D o: 1, ▁you: 13, ▁w an t: 1, ▁to: 33, ▁g et: 2, ▁be t ter: 2, ▁a t: 1, ▁mak ing: 2, ▁d e l ic i ou s: 1, ▁BBQ ?: 1, ▁ Y ou: 1, ▁will: 6, ▁have: 4, ▁the: 31, 

The implementation of BPE's code from the paper matches up pretty well with the library itself! The differences are probably accounted for by the `vocab_size`. There is also another technical difference in that in the SentencePiece implementation of BPE a priority queue is used to more efficiently keep track of the *best pairs*. Actually, there is a priority queue in the Python standard library called `heapq` if you would like to give that a try below! 

## Optionally try to implement BPE using a priority queue below

In [25]:
from heapq import heappush, heappop

def heapsort(iterable):
    """Sort an iterable using the heapsort algorithm."""
    
    h = []  # Initialize an empty list to use as a heap
    
    # Iterate over all values in the input iterable
    for value in iterable:
        heappush(h, value)                      # Push each value onto the heap to maintain the heap property

    # Pop all elements from the heap and return them in sorted order
    return [heappop(h) for i in range(len(h))]  # Extract the smallest elements one by one

In [26]:
a = [1, 4, 3, 1, 3, 2, 1, 4, 2]  # Initialize the list to be sorted

# Sort the list `a` using the heapsort function
sorted_a = heapsort(a)

print(sorted_a)  # Output: [1, 1, 1, 2, 2, 3, 3, 4, 4]

[1, 1, 1, 2, 2, 3, 3, 4, 4]

For a more extensive example consider looking at the [SentencePiece repo](https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb). The last few sections of this code were repurposed from that tutorial. Thanks for your participation! Next stop BERT and T5!